In [1]:
from telethon import TelegramClient
from datetime import datetime, timedelta, timezone
from api import *
import emoji
from transformers import pipeline
summarizer = pipeline('summarization', device=0)

In [36]:
async def get_messages(chat_name):
    global api_id
    global api_hash
    messages = []
    async with TelegramClient(api_id=api_id, api_hash=api_hash, session='this') as client:
            async for message in client.iter_messages(chat_name, offset_date=datetime.now(tz=timezone.utc) - timedelta(hours=24), reverse=True):
                messages.append(message)
    return messages

async def post_message(message):
    global api_id
    global api_hash
    async with TelegramClient(api_id=api_id, api_hash=api_hash, session='this') as client:
        await client.send_message(entity=-1002016594072, message=message)

In [52]:
def clean_messages(messages):

    """This is a helper function for create_summary which takes in a set of messages return from Telegram and
      returns those messages text contents as a single long string"""

    messages = [message.to_dict() for message in messages]
    texts = [emoji.demojize(message['message'].split('ORIGINAL')[0].replace('TRANSLATION :', '').replace('\n', ' ')) for message in messages]
    clean_text = ' '.join(texts)

    return clean_text


def summarize(input):
    """This is a helper function for create_summary which takes in a long string of text and returns a summarized string"""

    split_input = []

    start = 0
    end = 1024

    for i in range(len(input)//1024):
        split_input.append(input[start:end])
        start = end
        end += 1024

    split_input.append(input[end:])

    outputs = summarizer(split_input)

    summary = ' '.join([elt['summary_text'] for elt in outputs])

    return summary


def create_summary(messages):
    """This function takes in a set of messages from Telegram and returns a summary of less than 4000 characters"""

    text = clean_messages(messages)

    while len(text) > 4000:
        
        text = summarize(text)

    summary = str(datetime.now()) + '\n\n' + '\n\n'.join(text.split('. '))

    return summary

In [57]:
import time

tic = time.perf_counter()
messages = await get_messages('CombatMessagesUA')
summary = create_summary(messages)
await post_message(summary)
toc = time.perf_counter()

elapsed = toc - tic

print(elapsed)

Your max_length is set to 142, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Your max_length is set to 142, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Your max_length is set to 142, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)


86.99425079999992
